# 잡알리오 공공기관 채용정보 내역 크롤링
- ToDo : 잡알리오에서 원하는 공공기관(또는 상세검색)의 채용정보 내역을 크롤링
- base url : https://job.alio.go.kr/recruit.do

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## 1. 채용정보 내역
- 상세조건에 맞는 채용정보 리스트에서 각 공고별 링크 크롤링
- 잡알리오의 채용정보 url 구조 : https://job.alio.go.kr/recruitview.do?idx=**[INDEX]**
    - **[INDEX]** 에 해당하는 데이터를 크롤링

In [2]:
# 상세조건을 선택한 채용공고 리스트 url
base_url = 'https://job.alio.go.kr/recruit.do?idx=274842&s_date=2019.05.01&e_date=2024.09.23&org_type=&org_name=C0459&search_type=&keyword=&order=REG_DATE&sort=DESC&pageSet=50&pageNo='
idx_list=[]

for i in range(1, 9): #페이지 수
    list_url = base_url + str(i)    # 페이지별로 조회
    response = requests.get(list_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for href in soup.find('table', class_='tbl type_03').find('tbody').find_all('tr'):
        temp = href.find('td', class_='left').find('a')['href']     # 각 채용공고 html에서 하이퍼링크 tag의 index 데이터 추출
        idx_tag = re.findall(r"'(.*?)'", temp)
        idx_list.append(idx_tag)

## 2. 채용 상세정보 크롤링
- 채용 상세정보 url : https://job.alio.go.kr/recruitview.do?idx=**[INDEX]**
- 필요한 상세정보에 해당하는 tag 추출
- 한 채용공고에 여러 직무가 있는 경우)
    - 직무별로 상세정보를 나눠서 수집
- 인원정보에 '명' 제외
- 수집 데이터 : columns=['채용제목', '등록일', '채용인원', '응시자격', '직무', '응시인원', '선발인원']

In [7]:
details_data = []
job_base_url = "https://job.alio.go.kr/recruitview.do?idx="     # 채용정보 상세정보 링크

cnt = 1 #진행상황 체크용 건수 : 채용공고마다 건수 체크

for idx in idx_list:
    
    job_url = job_base_url + idx[0]
    job_response = requests.get(job_url)
    job_soup = BeautifulSoup(job_response.content, 'html.parser')
    
    # 채용제목
    title = job_soup.find('p')['title'].strip()
    
     # 등록일
    date = job_soup.find('th', text='등록일').find_next('td').text.strip()
    
    # 채용인원
    emp_num = re.findall(r"(.*?)명", job_soup.find('th', text='채용인원').find_next('td').text.strip())[0]
    
    # 응시자격
    qualification = job_soup.find('h4', text='응시자격').find_next('p').get_text(separator='\n')  
    
    # 한 공고에 여러 직무를 채용하는 경우 별개의 row로 수집
    for t in job_soup.find('div', id='tab-2').find_all('table'):
        
        # 직무명
        duty = t.find('th', class_='interviewName').text 
        
        # 응시인원
        enroll = t.find_all('td')[2].text.strip() 
        if '명' in enroll:
            enroll = re.findall(r"(.*?)명", enroll)[0]  # 명 제외
        elif enroll == '-':
            enroll = 0
        
        # 최종인원
        final = t.find_all('td')[5].text.strip() 
        if '명' in final:
            final = re.findall(r"(.*?)명", final)[0] # 명 제외
        elif final == '-':
            final = 0
        
        details_data.append([title, date, emp_num, qualification, duty, enroll, final])
        
    cnt += 1
    
    # 진행상황 체크
    if cnt % 10 == 1:
        print('cnt: ', cnt, ' data_num', len(details_data))
        print(details_data[-1])
        print(job_url)

df = pd.DataFrame(details_data, columns=['채용제목', '등록일', '채용인원', '응시자격', '직무', '응시인원', '선발인원'])

# 현재 경로에 csv파일로 저장
df.to_csv('job_alio.csv', index=False)